In [1]:
import numpy as np
import cv2
import os
import math
import matplotlib.pyplot as plt
import skimage.morphology
from skimage import img_as_ubyte

from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin

In [2]:
# Gaps are whether Gaps between words OR Gaps within sub-word
# removing sub-word Gaps
def FilterGaps(G,L):
    
    G = np.array(G)
    L = np.array(L)
    
    IQR = 2
    
    # Filtering Depending On IQR ( Interquartile Range ) Value
    G = G[ L >= IQR ]
    L = L[ L >= IQR ]
    
    Mean = np.mean(L)
    G = G[ L >= Mean ]
    L = L[ L >= Mean ]
    
    
    return G,L

In [3]:
def FindGaps(Line):
    
    G = []
    L = []

    # Vertical Projection Of the text
    VP = np.sum(Line,axis=0)   
    VP2 = VP[VP!=0].astype(np.int64)
    MFV = np.bincount(VP2).argmax()
    #print(VP2)
    #print(MFV)
    flag = 0
    for i in range(len(VP)):
        
        if VP[i] == 0 :
            if flag == 0:
                # This is a Gap Start 
                G.append(i)
                L.append(1)
                flag = 1
            else:
                # Increment Gap Length
                L[-1] +=1
            
        
        elif flag == 1:
            # End of The Gap
            flag = 0
        
    
    return G,L,MFV

In [4]:
def WordSegment(Line):
    
    # G => Array of Gaps, L => Length of each gap
    G,L,MFV = FindGaps(Line)
    G,L = FilterGaps(G,L)
    
    # Separete Each Word in a single Image
    Words = [ Line[0:,G[i]+L[i]:G[i+1]] for i in range(len(G)-1) ]    
    
    return Words,MFV

In [5]:
def ToBinaryImg(Img):
    ImgGray = cv2.cvtColor(Img, cv2.COLOR_BGR2GRAY)
    BinaryImg = np.array(ImgGray)
    BinaryImg[ ImgGray > 127 ] = 0
    BinaryImg[ ImgGray <= 127 ] = 1
    
    return BinaryImg

In [6]:
def MaxTrans(Line,Baseline):
    
    MTI   = 0
    MaxVal   = 0
    
    for i in range(0,Baseline):
        Trans = 0
        for k in range(1, len(Line[i]) ) :
            if Line[i][k] != Line[i][k-1]:
                Trans +=1
        if Trans > MaxVal:
            MaxVal = Trans
            MTI = i
    
    MTIidx = []
    
    #Flag = 0
    #for k in range(1, len(Line[MTI]) ):
    #    if Flag == 0 and Line[MTI][k] == 1 and Line[MTI][k] == 0 :
    #        MTIidx.append(k)
    #        Flag = 1
    #    elif Flag == 1
    
    return MTI ,np.array(MTIidx)

In [7]:
def DetectBaseline(Line):
    ThinnedImg = skimage.morphology.thin(Line)
    ThinnedImg = img_as_ubyte(ThinnedImg)
    
    HP = ThinnedImg
    
    Baseline  = np.array( np.sum(ThinnedImg,axis=1) )
    BaseIndex = np.argmax(Baseline) 
    
    return BaseIndex+2
    

In [13]:
def DetectCutPoints(Word,MTI,MFV):
    
    Flag = 0
    
    End = Start = -1
    Regions = []
    for i in range(len(Word[MTI])):
        if Flag == 0 and Word[MTI][i] == 1:
            End = i 
            Flag = 1

        elif Flag == 1 and Word[MTI][i] == 0:
            Start = i
            Mid = ( End + Start ) // 2
            #print(Mid)
            VP = np.sum(Word[End:Start+1],axis=0)
            print(VP)
            Cut = np.argwhere(VP == 0)
            # TODO : Choose Nearest K
            #rint(VP)
            #rint(Mid)
            if len(Cut) > 0:
                Cut = Cut[0]
                print("XXXXXXXx")
                print(Cut)
                
            
            elif  VP[Mid] == MFV :
                Cut = Mid
            
            else:
                VP2 = VP[End:Mid+1]
                Cut = np.argwhere( VP2 <= MFV )
                if len(Cut) > 0:
                    Cut = Cut[-1]
                else:
                    VP2 = VP[End:Start+1]
                    Cut = np.argwhere( VP2 <= MFV )
                    if len(Cut) > 0:
                        Cut = Cut[0]
                    else:
                        Cut = Mid
            
            Flag = 0
            print(Cut)
            Regions.append(Cut)
        #print(Regions)
            
                
    return Regions 
    

In [9]:
def Test(Word,Baseline,MFV):
    
    flag = 0
    VP = np.sum(Word,axis=0)
    
    Cuts = []
    for i in range(len(VP)):
        
        if VP[i] == MFV :
            print(VP[i])
            Cuts.append(i)
        elif flag == 1 and VP[i] == MFV:
            flag = 0
    
    return np.array(Cuts)
    

In [9]:
Img = cv2.imread('3.png')
dim = Img.shape

BinaryImg = ToBinaryImg(Img)

# Segment Words
Words,MFV = WordSegment(BinaryImg)


BaseIndex   = DetectBaseline(BinaryImg)
MTI,_ = MaxTrans(BinaryImg,BaseIndex)   
#Cuts = DetectCutPoints(Words[0],MTI,MFV)

#Cuts = Test(Words[0],BaseIndex,MFV)
#for i in range(len(Cuts)):
#    Img= cv2.rectangle(Img, (Cuts[i],0), (Cuts[i],dim[1]), (0,255,0), 1)


#Img = cv2.rectangle(Img,(0,BaseIndex),(dim[1],BaseIndex),(0,255,0),1)
#Img = cv2.rectangle(Img,(0,MaxTransIdx),(dim[1],MaxTransIdx),(0,255,0),1)

#cv2.imshow('BaseLine',BinaryImg)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


#for i in range(len(MTIidx)):
#    Img= cv2.rectangle(Img, (MTIidx[i],0), (MTIidx[i],dim[1]), (0,255,0), 1)
#BinaryImg = BinaryImg.astype(np.uint8)
#print(BinaryImg[ BinaryImg != 0 ])
#Img = img_as_ubyte(thin(BinaryImg,1000000))

cv2.imwrite('res1.jpg',Img)
cv2.imshow('BaseLine',BinaryImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

#cv2.imwrite('res1.jpg',Img)   
#for x in Words[::-1]:
#    cv2.imshow('Test Image',x)
#    cv2.waitKey(0)
#    cv2.destroyAllWindows()
#print(blackAndWhiteImage)


In [26]:
#For Code Testing

#arr = np.array([1,2,3,4,0,8,9,0])

#idx = np.argwhere(arr==0)


#if len(idx) > 0:
#    print(arr[idx[1]])
#else:
#    print("y")
#np.bincount(np.array([0, 1, 1, 3, 2, 1, 7]))
#Line = np.array([[0,2,3],[0,5,6],[0,7,8]])
#Line = np.sum(Line,axis=1)
#print(Line)

#VP = [ int(np.sum(Line[:,j] ) == 0)  for j in range( len(Line[0]) ) ]

#print(VP)

#x = np.array([1,1,1,0,0,1,1,1,0])

#(~x[:-1] & x[1:]).sum()
#np.count_nonzero(~x[:-1] & x[1:])


#G = np.array([1,2,3,4,5,6,7,8,9])
#G = G[ G < 5 ]
#print(G)
#skimage.morphology.thin
x = np.array(["hey","hel","hel","hey"])
np.bincount(x).argmax()

ValueError: invalid literal for int() with base 10: 'hey'